In [5]:
import numpy as np

In [13]:
class SineCosineOptimization():
    def __init__(self):
        ...
        
    def compile(self,obj,dimension,lb,ub,a=2,population_size=100,max_iter=100,optimization="maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.a=a
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    def __initialize_population(self,population_size,dim,lb,ub):
        return lb+np.random.rand(population_size,dim)*(ub-lb)
    
    def __update_R(self,current_iter,max_iter,a):
        r1 = a-a*(current_iter/max_iter)
        r2 = 2*np.pi*np.random.rand()
        r3 = np.random.rand()
        r4 = np.random.rand()
    
        return r1,r2,r3,r4
    
    def __update_population(self,populations,fittest_candidate,r1,r2,r3,r4,lb,ub):
        if r4 < 0.5:
            populations = populations + r1*np.sin(r2)*np.abs(r3*fittest_candidate-populations)
        else:
            populations = populations + r1*np.cos(r2)*np.abs(r3*fittest_candidate-populations)
        
        return np.clip(populations,lb,ub)
    
    def calculate_optimal_value(self):

        #Initialize the variable for  fittest candidate
        fittest_candidate=None
        candidate_evaluations = np.empty([self.population_size])

        #Initialize the random population
        populations = self.__initialize_population(self.population_size,self.dimension,self.lb,self.ub)

        ## This is the iteration loop.
        for i in range(self.max_iter):

            ## This loop will go through the all population of candidate and process them one by one
            for j in range(populations.shape[0]):
                candidate_evaluations[j]=self.objective_function(populations[j,:])


            ## Determining the fittest candidate
            if self.optimization == "maximize":
                fittest_indice = np.argmax(candidate_evaluations)
            elif self.optimization == "minimize":
                fittest_indice = np.argmin(candidate_evaluations)


            ## Updating the fittest candidate

            if fittest_candidate == None:
                fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
            else:
                current_fittest = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
                if self.optimization == "maximize" and fittest_candidate[1] < current_fittest[1]:
                    fittest_candidate = current_fittest

                if self.optimization == "minimize" and fittest_candidate[1] > current_fittest[1]:
                    fittest_candidate = current_fittest

            ## Updating the value of r1,r2,r3, and r4
            r1,r2,r3,r4 = self.__update_R(i,self.max_iter,self.a)

            ## Update the population
            
            for index,value in enumerate(populations):
                populations[index:] = self.__update_population(value,fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)

    

        return fittest_candidate
    
    def one_step_update(self,populations,pop_id,iteration,fittest_candidate=None):
        
        # Initialize candidate evaluation variables
        candidate_evaluations = np.empty([self.population_size])
        
        # Calculate the candidate evaluation using objective function
        for j in range(populations.shape[0]):
            candidate_evaluations[j]=self.objective_function(populations[j,:])
        
         ## Determining the fittest candidate
        if self.optimization == "maximize":
            fittest_indice = np.argmax(candidate_evaluations)
        elif self.optimization == "minimize":
            fittest_indice = np.argmin(candidate_evaluations)
        if fittest_candidate == None:
            fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
        
        r1,r2,r3,r4 = self.__update_R(iteration,self.max_iter,self.a)
        
        populations[pop_id,:] = self.__update_population(populations[pop_id,:].reshape(1,-1),fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)
        
        return populations
        

### Testing the Algorithm

In [25]:
## Defining a simple objective function
def obj(X):
    sums=0
    for i in X:
        sums = sums+i*i
    return sums
dim = 30
lb = np.array([-100]).repeat(dim)
ub = np.array([100]).repeat(dim)

##

In [26]:
max_iter=100
population_size=100
model = SineCosineOptimization()
model.compile(obj,dim,lb,ub,max_iter=max_iter,population_size=population_size,optimization="minimize")
answer = model.calculate_optimal_value()
print(answer)

(array([5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044,
       5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044,
       5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044,
       5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044,
       5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044,
       5.69472044, 5.69472044, 5.69472044, 5.69472044, 5.69472044]), 1.6890598067711703e-190)


In [9]:
def sums(X):
    return X[0]+X[1]
lb = np.array([-10,-10])
ub = np.array([10,10])
dimension = 2
max_iter=50
population_size=100
model = SineCosineOptimization()
model.compile(sums,dimension,lb,ub,max_iter=max_iter,population_size=population_size)
answer = model.calculate_optimal_value()
print(answer)

(array([10., 10.]), 20.0)
